<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/32Bits__FineTuning_Llama_SinteticData_PubHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Generar datos sintéicos

!pip install openai==0.28

prompt = "Genera un conjunto de datos de preguntas y respuestas sobre consejos de carrera profesional. Cada ejemplo debe tener el siguiente formato:\n\n<s>[INST] <pregunta> [/INST] <respuesta> </s>\n\nDonde <pregunta> es una pregunta relacionada con consejos de carrera, y <respuesta> es la respuesta detallada correspondiente."
temperature = .4
number_of_examples = 100


import os
import openai
import random
import pandas as pd
from sklearn.model_selection import train_test_split

openai.api_key = "API KEY"

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating a career advice question-answering dataset. Each example should have the following format:\n\n<s>[INST] <question> [/INST] <answer> </s>\n\nWhere <question> is a career-related question, and <answer> is the corresponding detailed answer.\n\nHere is the first question:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        prev_example = random.choice(prev_examples)
        if "<s>" in prev_example and "</s>" in prev_example:
            parts = prev_example.split("\n")
            if len(parts) == 3:
                question, answer = parts[1].split("<s>")[1].split("</s>")[0], parts[2]
                messages.append({
                    "role": "assistant",
                    "content": f"<s>[INST] {question} [/INST] {answer}"
                })

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
dataset = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    dataset.append(example)
    prev_examples.append(example)

# Create a DataFrame from the dataset
df = pd.DataFrame(dataset, columns=['text'])

# Save the dataset to a CSV file
df.to_csv('train.csv', index=False)

print("Dataset generated and saved successfully!")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
Generating example 20
Generating example 21
Generating example 22
Generating example 23
Generating example 24
Generating example 25
Generating example 26
Generating example 27
Generating example 28
Generating example 29
Generating example 30
Generating example 31
Generating example 32
Generating example 33
Generating example 34
Generating example 35
Generating example 36
Generating example 37
Generating example 38
Generating example 39
Generating example 40
Generating example 41
Genera

In [ ]:
#Instalar librerias
!pip install -U bitsandbytes accelerate peft transformers trl pandas datasets

In [ ]:
#Instalar librerias más pequeñas
!pip install -q accelerate==0.21.0 peft==0.4.0 transformers==4.31.0 trl==0.4.7 pandas datasets
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00


In [ ]:
# Entrenar el modelo
import os
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# El modelo que deseas entrenar desde el Hugging Face Hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Nombre del archivo CSV en Colab
dataset_file = "train.csv"

# Nombre del modelo ajustado
new_model = "llama-fine-tune_freebits"

# Cargar conjunto de datos desde el archivo CSV en Colab
dataset = pd.read_csv(dataset_file)

# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# TrainingArguments parameters
output_dir = "./results"
num_train_epochs = 1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}


In [ ]:
# Funciones
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

import pandas as pd
from sklearn.model_selection import train_test_split

dataset_df = pd.read_csv(dataset_file)
train_df, test_df = train_test_split(dataset_df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Entrenar el modelo
trainer.train()

# Guardar el modelo entrenado
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
# Inference
logging.set_verbosity(logging.CRITICAL)

prompt = "Escribe una noticia en español de 100 palabras sobre recomendaciones para buscar empleo"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] Escribe una noticia en español de 100 palabras sobre recomendaciones para buscar empleo [/INST]  ¡Claro! Aquí te dejo una noticia en español de 100 palabras sobre recomendaciones para buscar empleo:

Si estás buscando empleo, es importante que te prepares adecuadamente. En primer lugar, define claramente tus objetivos y habilidades para enfocar tu búsqueda. Luego, actualiza tu currículum vitae y redacta una breve descripción de tus habilidades y experiencia laboral. Asegúrate de personalizar tu solicitud para cada empresa y no te quedes sin hacer una buena impresión en las entrevistas. Por último, mantén una actitud positiva y persevera en tu búsqueda de empleo. ¡Buena suerte!


In [ ]:
#HuggingFaceLogin
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#PushModeltoHub
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.21G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CamiloVega/llama-fine-tune_freebits/commit/53a0eb5658182872db5e2ca7fca54023e0dca229', commit_message='Upload tokenizer', commit_description='', oid='53a0eb5658182872db5e2ca7fca54023e0dca229', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#Instalar librerias más pequeñas
!pip install -q accelerate==0.21.0 peft==0.4.0 transformers==4.31.0 trl==0.4.7 pandas datasets
!pip install -U bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("CamiloVega/Llama-Jobs-Tips")

# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained("CamiloVega/Llama-Jobs-Tips")

# Ejemplo de uso
prompt = "Escribe una noticia en español de 100 palabras sobre recomendaciones para buscar empleo"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])